In [ ]:
!git clone -b train --single-branch https://github.com/MLOpshub/extractive-qa-mlops.git
%cd extractive-qa-mlops

!pip -q install uv
!uv sync


In [ ]:
from google.colab import drive
drive.mount("/content/drive")


In [ ]:
!cd /content/extractive-qa-mlops
!pip -q install -e .

In [ ]:
import os

os.environ["MODELS_DIR"] = "/content/drive/MyDrive/extractive-qa-mlops/artifacts/models"
os.environ["MLRUNS_DIR"] = "/content/drive/MyDrive/extractive-qa-mlops/mlruns"


Running sweep which includes the baseline traing

In [ ]:
!uv run python scripts/sweep.py --train configs/train.yaml --sweep configs/sweep.yaml

Running baseline training alone

In [ ]:
!uv run python -m extractive_qa_mlops.train --config configs/train.yaml


In [ ]:
!cd /content/extractive-qa-mlops && uv run python -m extractive_qa_mlops.eval_cli \
  --model_dir "/content/drive/MyDrive/extractive-qa-mlops/artifacts/models/best" \
  --dataset squad --samples 500


MLflow UI

```powershell
uv run python -m mlflow ui --backend-store-uri "file:./mlruns" --port 5000
```